#  Data Preprocessing Pipeline
# **Purpose:** Clean and prepare raw data for feature engineering and modeling


In [7]:
#Setup and imports
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')


In [8]:
# Utilities
import os
from datetime import datetime
import pickle

In [9]:
# Display settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.4f}'.format)

print(f"Preprocessing started at: {datetime.now()}")


Preprocessing started at: 2025-06-14 21:56:42.660103


# 2. Data Loading and Initial Inspection

In [10]:
# Load raw data
df_raw = pd.read_csv('/home/joojo/Desktop/azubia_test/data/raw/bank-full.csv', delimiter=';')

print(f"Raw data shape: {df_raw.shape}")
print(f"Memory usage: {df_raw.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

# %%
# Basic data inspection
print("=== DATA OVERVIEW ===")
print(df_raw.info())
print("\n=== FIRST 5 ROWS ===")
display(df_raw.head())
print("\n=== LAST 5 ROWS ===")
display(df_raw.tail())

Raw data shape: (45211, 17)
Memory usage: 25.75 MB
=== DATA OVERVIEW ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        45211 non-null  int64 
 1   job        45211 non-null  object
 2   marital    45211 non-null  object
 3   education  45211 non-null  object
 4   default    45211 non-null  object
 5   balance    45211 non-null  int64 
 6   housing    45211 non-null  object
 7   loan       45211 non-null  object
 8   contact    45211 non-null  object
 9   day        45211 non-null  int64 
 10  month      45211 non-null  object
 11  duration   45211 non-null  int64 
 12  campaign   45211 non-null  int64 
 13  pdays      45211 non-null  int64 
 14  previous   45211 non-null  int64 
 15  poutcome   45211 non-null  object
 16  y          45211 non-null  object
dtypes: int64(7), object(10)
memory usage: 5.9+ MB
None

=== FIRST 5 ROWS ===

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143,yes,no,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,no,29,yes,no,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,no,2,yes,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,no,1506,yes,no,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,no,1,no,no,unknown,5,may,198,1,-1,0,unknown,no



=== LAST 5 ROWS ===


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
45206,51,technician,married,tertiary,no,825,no,no,cellular,17,nov,977,3,-1,0,unknown,yes
45207,71,retired,divorced,primary,no,1729,no,no,cellular,17,nov,456,2,-1,0,unknown,yes
45208,72,retired,married,secondary,no,5715,no,no,cellular,17,nov,1127,5,184,3,success,yes
45209,57,blue-collar,married,secondary,no,668,no,no,telephone,17,nov,508,4,-1,0,unknown,no
45210,37,entrepreneur,married,secondary,no,2971,no,no,cellular,17,nov,361,2,188,11,other,no


In [11]:
# Column names and data types
print("=== COLUMN INFORMATION ===")
col_info = pd.DataFrame({
    'Column': df_raw.columns,
    'Data_Type': df_raw.dtypes,
    'Non_Null_Count': df_raw.count(),
    'Null_Count': df_raw.isnull().sum(),
    'Null_Percentage': (df_raw.isnull().sum() / len(df_raw) * 100).round(2)
})
display(col_info)

=== COLUMN INFORMATION ===


,Column,Data_Type,Non_Null_Count,Null_Count,Null_Percentage
age,age,int64,45211,0,0.0000
job,job,object,45211,0,0.0000
marital,marital,object,45211,0,0.0000
education,education,object,45211,0,0.0000
default,default,object,45211,0,0.0000
balance,balance,int64,45211,0,0.0000
housing,housing,object,45211,0,0.0000
loan,loan,object,45211,0,0.0000
contact,contact,object,45211,0,0.0000
day,day,int64,45211,0,0.0000


# 3. Data Quality Assessment

In [12]:
# Create a copy for processing
df = df_raw.copy()

print("=== INITIAL DATA QUALITY REPORT ===")
print(f"Total rows: {len(df):,}")
print(f"Total columns: {len(df.columns)}")
print(f"Duplicate rows: {df.duplicated().sum():,}")
print(f"Completely empty rows: {df.isnull().all(axis=1).sum():,}")

=== INITIAL DATA QUALITY REPORT ===
Total rows: 45,211
Total columns: 17
Duplicate rows: 0
Completely empty rows: 0


In [13]:
# Check for inconsistent data types
print("=== DATA TYPE CONSISTENCY CHECK ===")
for col in df.columns:
    unique_types = df[col].apply(type).value_counts()
    if len(unique_types) > 1:
        print(f"Column '{col}' has mixed data types:")
        print(unique_types)
        print()



=== DATA TYPE CONSISTENCY CHECK ===


In [14]:
# Identify categorical vs numerical columns
numerical_cols = df.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
datetime_cols = df.select_dtypes(include=['datetime64']).columns.tolist()

print(f"Numerical columns ({len(numerical_cols)}): {numerical_cols}")
print(f"Categorical columns ({len(categorical_cols)}): {categorical_cols}")
print(f"Datetime columns ({len(datetime_cols)}): {datetime_cols}")


Numerical columns (7): ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
Categorical columns (10): ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'y']
Datetime columns (0): []


# 4. Handle Missing Values

In [15]:
# Detailed missing value analysis
print("=== MISSING VALUE ANALYSIS ===")
missing_data = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2),
    'Data_Type': df.dtypes
})
missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)
display(missing_data)


=== MISSING VALUE ANALYSIS ===


,Column,Missing_Count,Missing_Percentage,Data_Type


In [16]:
# Missing value patterns
if len(missing_data) > 0:
    print("=== MISSING VALUE PATTERNS ===")
    # Check if missing values are correlated
    missing_pattern = df.isnull()
    missing_combinations = missing_pattern.value_counts().head(10)
    print("Top 10 missing value combinations:")
    print(missing_combinations)
else:
    print("No missing values found in the dataset!")

No missing values found in the dataset!


#5. Handle Duplicates

In [17]:
print("=== DUPLICATE HANDLING ===")
initial_rows = len(df)
duplicates = df.duplicated().sum()

print(f"Total duplicate rows: {duplicates:,}")

=== DUPLICATE HANDLING ===


Total duplicate rows: 0


# 6. Data Type Optimization

In [18]:
print("=== DATA TYPE OPTIMIZATION ===")
memory_before = df.memory_usage(deep=True).sum() / 1024**2
print(f"Memory usage before optimization: {memory_before:.2f} MB")

# Optimize numerical columns
for col in numerical_cols:
    if col in df.columns:  # Check if column still exists after preprocessing
        col_min = df[col].min()
        col_max = df[col].max()

        # Integer optimization
        if df[col].dtype in ['int64', 'int32']:
            if col_min >= 0:  # Unsigned integers
                if col_max < 255:
                    df[col] = df[col].astype(np.uint8)
                elif col_max < 65535:
                    df[col] = df[col].astype(np.uint16)
                elif col_max < 4294967295:
                    df[col] = df[col].astype(np.uint32)
            else:  # Signed integers
                if col_min > -128 and col_max < 127:
                    df[col] = df[col].astype(np.int8)
                elif col_min > -32768 and col_max < 32767:
                    df[col] = df[col].astype(np.int16)
                elif col_min > -2147483648 and col_max < 2147483647:
                    df[col] = df[col].astype(np.int32)

        # Float optimization
        elif df[col].dtype in ['float64']:
            df[col] = pd.to_numeric(df[col], downcast='float')

=== DATA TYPE OPTIMIZATION ===


Memory usage before optimization: 25.75 MB


In [19]:
# Optimize categorical columns
for col in categorical_cols:
    if col in df.columns:
        unique_count = df[col].nunique()
        total_count = len(df)

        # Convert to category if it saves memory
        if unique_count / total_count < 0.5:
            df[col] = df[col].astype('category')
            print(f"Converted '{col}' to category ({unique_count} unique values)")

memory_after = df.memory_usage(deep=True).sum() / 1024**2
print(f"\nMemory usage after optimization: {memory_after:.2f} MB")
print(f"Memory saved: {memory_before - memory_after:.2f} MB ({((memory_before - memory_after) / memory_before * 100):.1f}%)")

Converted 'job' to category (12 unique values)
Converted 'marital' to category (3 unique values)
Converted 'education' to category (4 unique values)
Converted 'default' to category (2 unique values)
Converted 'housing' to category (2 unique values)
Converted 'loan' to category (2 unique values)
Converted 'contact' to category (3 unique values)
Converted 'month' to category (12 unique values)
Converted 'poutcome' to category (4 unique values)
Converted 'y' to category (2 unique values)

Memory usage after optimization: 1.00 MB
Memory saved: 24.75 MB (96.1%)


# 7. Outlier Detection and Documentation

In [20]:
print("=== OUTLIER DETECTION ===")
outlier_summary = {}

for col in numerical_cols:
    if col in df.columns and df[col].dtype in [np.number]:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
        outlier_count = len(outliers)
        outlier_percentage = (outlier_count / len(df)) * 100

        outlier_summary[col] = {
            'count': outlier_count,
            'percentage': outlier_percentage,
            'lower_bound': lower_bound,
            'upper_bound': upper_bound,
            'min_value': df[col].min(),
            'max_value': df[col].max()
        }

        if outlier_count > 0:
            print(f"{col}: {outlier_count:,} outliers ({outlier_percentage:.2f}%)")
            print(f"  Range: [{df[col].min():.2f}, {df[col].max():.2f}]")
            print(f"  Normal range: [{lower_bound:.2f}, {upper_bound:.2f}]")
            print()

# Note: We're documenting outliers but not removing them at this stage
# Outlier treatment should be done during feature engineering based on domain knowledge


=== OUTLIER DETECTION ===


# 8. Basic Data Validation

In [21]:
print("=== DATA VALIDATION CHECKS ===")
validation_issues = []

# Check for negative values where they shouldn't exist
for col in ['age', 'duration', 'campaign', 'pdays']:
    if col in df.columns:
        negative_count = (df[col] < 0).sum()
        if negative_count > 0:
            validation_issues.append(f"{col}: {negative_count} negative values")
            print(f"WARNING: {col} has {negative_count} negative values")


=== DATA VALIDATION CHECKS ===


In [22]:
# Check for unrealistic age values
if 'age' in df.columns:
    unrealistic_age = ((df['age'] < 18) | (df['age'] > 100)).sum()
    if unrealistic_age > 0:
        validation_issues.append(f"age: {unrealistic_age} unrealistic values (<18 or >100)")
        print(f"WARNING: {unrealistic_age} unrealistic age values found")

In [23]:
# Check for zero duration (might be data quality issue)
if 'duration' in df.columns:
    zero_duration = (df['duration'] == 0).sum()
    if zero_duration > 0:
        print(f"INFO: {zero_duration} records with zero duration")

INFO: 3 records with zero duration


In [24]:
# Check target variable distribution
if 'y' in df.columns:
    target_dist = df['y'].value_counts()
    print(f"\nTarget variable distribution:")
    print(target_dist)
    print(f"Target balance: {target_dist.min()/target_dist.max():.3f}")

if not validation_issues:
    print("✓ No major data validation issues found!")
else:
    print(f"⚠ Found {len(validation_issues)} validation issues to review")


Target variable distribution:
y
no     39922
yes     5289
Name: count, dtype: int64
Target balance: 0.132
⚠ Found 1 validation issues to review


# 9. Final Data Summary

In [25]:
print("=== FINAL PREPROCESSED DATA SUMMARY ===")
print(f"Final dataset shape: {df.shape}")
print(f"Memory usage: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Missing values: {df.isnull().sum().sum()}")
print(f"Duplicate rows: {df.duplicated().sum()}")

print("\n=== COLUMN SUMMARY ===")
final_summary = pd.DataFrame({
    'Column': df.columns,
    'Data_Type': df.dtypes,
    'Non_Null_Count': df.count(),
    'Unique_Values': [df[col].nunique() for col in df.columns],
    'Memory_MB': df.memory_usage(deep=True)[1:] / 1024**2
})
display(final_summary)


=== FINAL PREPROCESSED DATA SUMMARY ===
Final dataset shape: (45211, 17)
Memory usage: 1.00 MB
Missing values: 0
Duplicate rows: 0

=== COLUMN SUMMARY ===


,Column,Data_Type,Non_Null_Count,Unique_Values,Memory_MB
age,age,uint8,45211,77,0.0431
job,job,category,45211,12,0.0441
marital,marital,category,45211,3,0.0434
education,education,category,45211,4,0.0435
default,default,category,45211,2,0.0433
balance,balance,int32,45211,7168,0.1725
housing,housing,category,45211,2,0.0433
loan,loan,category,45211,2,0.0433
contact,contact,category,45211,3,0.0434
day,day,uint8,45211,31,0.0431


In [26]:
# Quick statistical summary for numerical columns
numerical_cols_final = df.select_dtypes(include=[np.number]).columns.tolist()
if numerical_cols_final:
    print("\n=== NUMERICAL COLUMNS STATISTICS ===")
    display(df[numerical_cols_final].describe())


=== NUMERICAL COLUMNS STATISTICS ===


,age,balance,day,duration,campaign,pdays,previous
count,45211.0000,45211.0000,45211.0000,45211.0000,45211.0000,45211.0000,45211.0000
mean,40.9362,1362.2721,15.8064,258.1631,2.7638,40.1978,0.5803
std,10.6188,3044.7658,8.3225,257.5278,3.0980,100.1287,2.3034
min,18.0000,-8019.0000,1.0000,0.0000,1.0000,-1.0000,0.0000
25%,33.0000,72.0000,8.0000,103.0000,1.0000,-1.0000,0.0000
50%,39.0000,448.0000,16.0000,180.0000,2.0000,-1.0000,0.0000
75%,48.0000,1428.0000,21.0000,319.0000,3.0000,-1.0000,0.0000
max,95.0000,102127.0000,31.0000,4918.0000,63.0000,871.0000,275.0000


# 11. Preprocessing Log Summary

In [27]:
print("=== PREPROCESSING COMPLETION REPORT ===")
print(f"Started: {datetime.now()}")
print(f"Input: {df_raw.shape[0]:,} rows × {df_raw.shape[1]} columns")
print(f"Output: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"Rows removed: {df_raw.shape[0] - df.shape[0]:,} ({((df_raw.shape[0] - df.shape[0])/df_raw.shape[0]*100):.2f}%)")
print(f"Memory optimized: {memory_before - memory_after:.2f} MB saved")
print(f"\n✓ Data preprocessing completed successfully!")
print(f"✓ Ready for feature engineering and modeling")

# Next steps reminder
print("\n=== NEXT STEPS ===")
print("1. Feature Engineering (duration transformations, cyclical encoding, etc.)")
print("2. Exploratory Data Analysis (EDA)")
print("3. Feature Selection")
print("4. Model Development")
print("5. Model Evaluation")

=== PREPROCESSING COMPLETION REPORT ===
Started: 2025-06-14 21:56:48.222294
Input: 45,211 rows × 17 columns
Output: 45,211 rows × 17 columns
Rows removed: 0 (0.00%)
Memory optimized: 24.75 MB saved

✓ Data preprocessing completed successfully!
✓ Ready for feature engineering and modeling

=== NEXT STEPS ===
1. Feature Engineering (duration transformations, cyclical encoding, etc.)
2. Exploratory Data Analysis (EDA)
3. Feature Selection
4. Model Development
5. Model Evaluation


In [28]:
from scipy.stats import pointbiserialr

# Check correlation of all features with 'y'
print("=== CORRELATION WITH TARGET VARIABLE 'y' ===")

# For numerical features: use point biserial correlation

correlations = {}

# Convert target to binary (assuming 'yes' is positive class)
y_binary = (df['y'] == 'yes').astype(int)

for col in numerical_cols:
    if col in df.columns:
        corr, pval = pointbiserialr(df[col], y_binary)
        correlations[col] = corr
        print(f"{col}: correlation = {corr:.4f} (p={pval:.4g})")

# For categorical features: use Cramér's V
import scipy.stats as stats

def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

for col in categorical_cols:
    if col in df.columns and col != 'y':
        v = cramers_v(df[col], df['y'])
        correlations[col] = v
        print(f"{col}: Cramér's V = {v:.4f}")

# Optionally, display as a sorted DataFrame
corr_df = pd.DataFrame(list(correlations.items()), columns=['Feature', 'Correlation_with_y'])
corr_df = corr_df.sort_values('Correlation_with_y', key=np.abs, ascending=False)
display(corr_df)

=== CORRELATION WITH TARGET VARIABLE 'y' ===
age: correlation = 0.0252 (p=8.826e-08)
balance: correlation = 0.0528 (p=2.521e-29)
day: correlation = -0.0283 (p=1.654e-09)
duration: correlation = 0.3945 (p=0)
campaign: correlation = -0.0732 (p=1.012e-54)
pdays: correlation = 0.1036 (p=3.791e-108)
previous: correlation = 0.0932 (p=7.802e-88)
job: Cramér's V = 0.1351
marital: Cramér's V = 0.0656
education: Cramér's V = 0.0722
default: Cramér's V = 0.0217
housing: Cramér's V = 0.1390
loan: Cramér's V = 0.0679
contact: Cramér's V = 0.1512
month: Cramér's V = 0.2598
poutcome: Cramér's V = 0.3116


,Feature,Correlation_with_y
3,duration,0.3945
15,poutcome,0.3116
14,month,0.2598
13,contact,0.1512
11,housing,0.1390
7,job,0.1351
5,pdays,0.1036
6,previous,0.0932
4,campaign,-0.0732
9,education,0.0722
